# AST debugging using interactive Go

This code was used in the original development of the AST parsing performed by the [Exercism Go Test Runner](https://github.com/exercism/go-test-runner/). It uses the [gophernotes project](https://github.com/gopherdata/gophernotes). Consult the gophernotes docs for installation instructions. Once installed, you should be able to execute and modify the following debug code "live".

In [3]:
package main

import (
  "bytes"
  "fmt"
  "go/ast"
  "go/parser"
  "go/printer"
  "go/token"
  "go/format"
  "log"
  "os"
  "os/exec"
  "os/user"
  "strings"
  "strconv"
  "golang.org/x/tools/go/ast/astutil"
  "github.com/exercism/go-test-runner/testrunner"  // uses the main branch of the published package
  // "testrunner"
  // Alternatively, comment out the github import and use your local copy by 
  // symlinking the testrunner directory from your local repo to $GOPATH/src
  // 
  // If you import the local package, after making a change 
  // you will need to stop the kernel and re-run this cell to rebuild the package
  // For a faster development cycle, copy the functions you are working on to a new cell
  // and call the local version instead
)

In [5]:
// Run the test command, view the output and test out the extraction code
cwd, err := os.Getwd()
if err != nil {
    fmt.Println(err)
}
input_dir := cwd + "/testdata/concept/conditionals"
output_dir := cwd + "/outdir"
testName := "TestParseCard"
testFile := cwd + "/testdata/concept/conditionals/conditionals_test.go"
testCodeSrc := testrunner.ExtractTestCode(testName, testFile)
goExe, err := exec.LookPath("go")
var stdout, stderr bytes.Buffer

  testCmd := &exec.Cmd{
    Dir:    input_dir,
    Path:   goExe,
    Args:   []string{goExe, "test", "--json", "."},
    Stdout: &stdout,
    Stderr: &stderr,
  }
  if err := testCmd.Run(); err != nil {                                         
    if exitError, ok := err.(*exec.ExitError); ok {                             
      if 1 == exitError.ExitCode() {                                            
        // `go test` returns 1 when tests fail                                  
        // The test runner should continue and return 0 in this case            
        log.Printf(                                                             
          "warning: ignoring exit code 1 from '%s'", testCmd.String(),          
        )                                                                       
      } else {                                                                  
        log.Fatalf("'%s' failed with exit code %d: %s",                         
          testCmd.String(), exitError.ExitCode(), err)                          
      }                                                                         
    }                                                                           
  }      
fmt.Println("go test --json output:\n", &stdout)
fmt.Println("\n------\n")
fmt.Println("extracted test function:\n", testCodeSrc)

go test --json output:
 {"Time":"2021-01-19T13:46:01.680968-06:00","Action":"run","Package":"conditionals","Test":"TestNonSubtest"}
{"Time":"2021-01-19T13:46:01.681299-06:00","Action":"output","Package":"conditionals","Test":"TestNonSubtest","Output":"=== RUN   TestNonSubtest\n"}
{"Time":"2021-01-19T13:46:01.681316-06:00","Action":"output","Package":"conditionals","Test":"TestNonSubtest","Output":"the whole block\n"}
{"Time":"2021-01-19T13:46:01.681321-06:00","Action":"output","Package":"conditionals","Test":"TestNonSubtest","Output":"should be returned\n"}
{"Time":"2021-01-19T13:46:01.681333-06:00","Action":"output","Package":"conditionals","Test":"TestNonSubtest","Output":"--- PASS: TestNonSubtest (0.00s)\n"}
{"Time":"2021-01-19T13:46:01.681337-06:00","Action":"pass","Package":"conditionals","Test":"TestNonSubtest","Elapsed":0}
{"Time":"2021-01-19T13:46:01.681351-06:00","Action":"run","Package":"conditionals","Test":"TestTODOSubtest"}
{"Time":"2021-01-19T13:46:01.681354-06:00","Actio

509 <nil>

In [6]:
// Output a pretty printed AST
fset := token.NewFileSet()
f, err := parser.ParseFile(fset, "", "package main\n" + testCodeSrc, parser.ParseComments)
if err != nil {
    log.Fatal(err)
}

fexp, err := parser.ParseExpr(testCodeSrc)
fmt.Println("ast:\n")
ast.Print(fset, fexp)

ast:

     0  *ast.FuncLit {
     1  .  Type: *ast.FuncType {
     2  .  .  Func: 1:1
     3  .  .  Params: *ast.FieldList {
     4  .  .  .  Opening: 1:6
     5  .  .  .  List: []*ast.Field (len = 1) {
     6  .  .  .  .  0: *ast.Field {
     7  .  .  .  .  .  Names: []*ast.Ident (len = 1) {
     8  .  .  .  .  .  .  0: *ast.Ident {
     9  .  .  .  .  .  .  .  NamePos: 2:6
    10  .  .  .  .  .  .  .  Name: "_"
    11  .  .  .  .  .  .  .  Obj: *ast.Object {
    12  .  .  .  .  .  .  .  .  Kind: var
    13  .  .  .  .  .  .  .  .  Name: "_"
    14  .  .  .  .  .  .  .  .  Decl: *(obj @ 6)
    15  .  .  .  .  .  .  .  }
    16  .  .  .  .  .  .  }
    17  .  .  .  .  .  }
    18  .  .  .  .  .  Type: *ast.SelectorExpr {
    19  .  .  .  .  .  .  X: *ast.Ident {
    20  .  .  .  .  .  .  .  NamePos: 2:10
    21  .  .  .  .  .  .  .  Name: "testing"
    22  .  .  .  .  .  .  .  Obj: *ast.Object {
    23  .  .  .  .  .  .  .  .  Kind: bad
    24  .  .  .  .  .  .  .  .  Name: ""
    25  

   205  .  .  .  .  .  .  .  .  .  .  }
   206  .  .  .  .  .  .  .  .  .  }
   207  .  .  .  .  .  .  .  .  }
   208  .  .  .  .  .  .  .  .  Rbrace: 16:3
   209  .  .  .  .  .  .  .  .  Incomplete: false
   210  .  .  .  .  .  .  .  }
   211  .  .  .  .  .  .  .  2: *ast.CompositeLit {
   212  .  .  .  .  .  .  .  .  Lbrace: 16:8
   213  .  .  .  .  .  .  .  .  Elts: []ast.Expr (len = 3) {
   214  .  .  .  .  .  .  .  .  .  0: *ast.KeyValueExpr {
   215  .  .  .  .  .  .  .  .  .  .  Key: *ast.Ident {
   216  .  .  .  .  .  .  .  .  .  .  .  NamePos: 16:13
   217  .  .  .  .  .  .  .  .  .  .  .  Name: "name"
   218  .  .  .  .  .  .  .  .  .  .  }
   219  .  .  .  .  .  .  .  .  .  .  Colon: 17:4
   220  .  .  .  .  .  .  .  .  .  .  Value: *ast.BasicLit {
   221  .  .  .  .  .  .  .  .  .  .  .  ValuePos: 18:1
   222  .  .  .  .  .  .  .  .  .  .  .  Kind: STRING
   223  .  .  .  .  .  .  .  .  .  .  .  Value: "\"parse king\""
   224  .  .  .  .  .  .  .  .  .  .  }
   225  .  .  .

   445  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Name: "Errorf"
   446  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  }
   447  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  }
   448  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Lparen: 26:50
   449  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Args: []ast.Expr (len = 4) {
   450  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  0: *ast.BasicLit {
   451  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  ValuePos: 27:1
   452  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Kind: STRING
   453  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Value: "\"ParseCard(%s) = %d, want %d\""
   454  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  }
   455  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  1: *ast.SelectorExpr {
   456  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  X: *ast.Ident {
   457  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  

In [7]:
// This code most closely resembles the implementation (as of jan 2021)
// It "manually" traverses the AST based on an expected structure
// defined by the spec - see the README for details: 
// https://github.com/exercism/go-test-runner/blob/master/README.md#subtest-format-specification
cwd, err := os.Getwd()
if err != nil {
    fmt.Println(err)
}
input_dir := cwd + "/testdata/concept/conditionals"
output_dir := cwd + "/outdir"
testName := "TestParseCard"
testFile := cwd + "/testdata/concept/conditionals/conditionals_test.go"
testCodeSrc := testrunner.ExtractTestCode(testName, testFile)
//fmt.Println(testCodeSrc)

fset := token.NewFileSet()
f, err := parser.ParseFile(fset, testFile, "package main\n" + testCodeSrc, parser.ParseComments)
if err != nil {
    log.Fatal(err)
}
//ast.Print(fset, f)

type testMeta struct {
    subTName    string             // name of subtest
    subTKey     string             // subtest name key
    origTDName  string             // original test data []struct name
    newTDName   string             // new test data struct name
    TD          []ast.Expr         // original tests data node
    subTest     ast.Stmt           // Run() function literal node
} 

metadata := testMeta{
    subTName: "parse queen",
}

fAST, ok := f.Decls[0].(*ast.FuncDecl)
if !ok {
    fmt.Println("subtest does not contain a function as the first declaration") 
}
fbAST := fAST.Body.List  // f.Decls[0].Body.List*/

if 2 != len(fbAST) {
    fmt.Println("subtests are constrained to two top level nodes") 
}

tdast, ok := fbAST[0].(*ast.AssignStmt) // f.Decls[0].Body.List[0]
if !ok {
    fmt.Println("subtest does not contain a test data assignment as the first node") 
}
rast, ok := fbAST[1].(*ast.RangeStmt)
if !ok {
    fmt.Println("subtest does not contain a range keyword as the second node") 
}

// Go to work on the test data assignment
lhs1, ok := tdast.Lhs[0].(*ast.Ident) // f.Decls[0].Body.List[0].Lhs[0]
if !ok {
    fmt.Println("subtest test data assignment not found") 
}
if ast.Var != lhs1.Obj.Kind {
    fmt.Println("subtest test data assignment not a var") 
}
metadata.origTDName = lhs1.Name

rhs1, ok := tdast.Rhs[0].(*ast.CompositeLit) // f.Decls[0].Body.List[0].Rhs[0]
if !ok {
    fmt.Println("subtest test data assignment not a composite literal") 
}


// Loop for all of the test data structs
for i, td := range rhs1.Elts {
    vals, ok := td.(*ast.CompositeLit)
    if ok {
        // Loop for each KeyValueExpr in the struct
        for _, tv := range vals.Elts {
            if kv, ok := tv.(*ast.KeyValueExpr); ok {
                if value, ok := kv.Value.(*ast.BasicLit); ok {
                    // [TODO] https://github.com/golang/go/blob/f1980efb92c011eab71aa61b68ccf58d845d1de7/src/testing/match.go#L50
                    altSubTName := strings.Replace(metadata.subTName, " ", "_", -1)
                    if token.STRING == value.Kind && (strconv.Quote(metadata.subTName) == value.Value || strconv.Quote(altSubTName) == value.Value) {
                        // Store the parent array of KeyValueExprs 
                        //  - this is the test data element for the requested subtest
                        metadata.TD = vals.Elts
                        // Store the subtest data "name" value
                        metadata.subTKey = kv.Key.(*ast.Ident).Name
                    }
                }
            }
        }
    }
}

// Go to work on the range statement

// Confirm that the range is over the test data
if (rast.X.(*ast.Ident).Name != metadata.origTDName) {
    fmt.Printf("mismatch between test data (%s) and range value (%s)", rast.X.(*ast.Ident).Name, metadata.origTDName)
}

// Pull the name of the subtest data being used
metadata.newTDName = rast.Value.(*ast.Ident).Name

// Parse out the Run() call within the range statement
rblexp := rast.Body.List[0].(*ast.ExprStmt).X

// Parse out the function literal from the Run() call within the range statement
runcall := rblexp.(*ast.CallExpr).Fun

if ("Run" != runcall.(*ast.SelectorExpr).Sel.Name) {
    fmt.Printf("Run() call did not follow range loop: (%s)", runcall.(*ast.SelectorExpr).Sel.Name)
}

runselector := rblexp.(*ast.CallExpr).Args[0]
runfunclit := rblexp.(*ast.CallExpr).Args[1]


if (metadata.newTDName != runselector.(*ast.SelectorExpr).X.(*ast.Ident).Name) {
    fmt.Printf("Run() call not passing expected test data %s: %s", metadata.newTDName, runselector.(*ast.SelectorExpr).X.(*ast.Ident).Name)
}

if (metadata.subTKey != runselector.(*ast.SelectorExpr).Sel.Name) {
    fmt.Printf("Run() call name (%s) does not match name from test data struct: %s", runselector.(*ast.SelectorExpr).X.(*ast.Ident).Name, metadata.subTKey)
}

body := runfunclit.(*ast.FuncLit).Body.List[0]
metadata.subTest = body

// rhs1, ok := tdast.Rhs[0].(*ast.CompositeLit) // f.Decls[0].Body.List[0].Rhs[0]

rhstype := rhs1.Type.(*ast.ArrayType) //Elt.(*ast.StructType)
//rhsstruct := rhstype.Elt
//rhs1.Type = rhsstruct

rhs1.Type = rhs1.Type.(*ast.ArrayType).Elt

fmt.Printf("rhstypelt %T | %+v\n", rhstype, rhstype)

rhs1.Elts = metadata.TD
//tdast.Rhs = metadata.TD

/*  69  .  .  .  .  .  .  Rhs: []ast.Expr (len = 1) {
    70  .  .  .  .  .  .  .  0: *ast.CompositeLit {
    71  .  .  .  .  .  .  .  .  Type: *ast.ArrayType {
    72  .  .  .  .  .  .  .  .  .  Lbrack: ~/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:3:11
    73  .  .  .  .  .  .  .  .  .  Elt: *ast.StructType {*/


lhs1.Name = metadata.newTDName

assgn := &ast.AssignStmt{
    Lhs: []ast.Expr{lhs1},
    TokPos: tdast.TokPos,
    Tok:    tdast.Tok,
    Rhs: []ast.Expr{rhs1},
}
tdast = assgn

fbAST[1] = metadata.subTest

fmt.Printf("%+v\n--\n", metadata)
//fmt.Printf("%T | %+v\n", rhs.Elts, rhs.Elts)

var buf bytes.Buffer
fmt.Printf("type %T: %s", metadata.TD, buf.String())
if err := format.Node(&buf, fset, f); err != nil {
    panic(err)
}
fmt.Printf("output node: %s", buf.String())

Run() call name (tt) does not match name from test data struct: rhstypelt *ast.ArrayType | &{Lbrack:59 Len:<nil> Elt:0xc000844d80}
{𒀸subTName:parse queen 𒀸subTKey: 𒀸origTDName:tests 𒀸newTDName:tt TD:[] 𒀸subTest:0xc00083f6c0}
--
type []ast.Expr: output node: package main

func TestParseCard(t *testing.T) {
	tt := struct {
		name string
		card string
		want int
	}{}

	if got := ParseCard(tt.card); got != tt.want {
		t.Errorf("ParseCard(%s) = %d, want %d", tt.card, got, tt.want)
	}

}


242 <nil>

In [8]:
// This code uses the Apply function to iterate over the AST
// Both Apply and Inspect might be useful in providing more flexible parsing code,
// not reliant on a simple subtest meeting the spec (see link in the cell above)
astutil.Apply(fexp, func(cr *astutil.Cursor) bool {
    n := cr.Node()
    //n := cr.Parent()
    switch x := n.(type) {
    case *ast.BasicLit:
        if token.STRING == x.Kind && "\"parse queen\"" == x.Value{
            fmt.Printf("bl %s:\t%s - %s\n", fset.Position(n.Pos()), x.Kind, x.Value)
            //tdAssgn = cr.Parent()
            //tdPNode = cr.Parent()
            //return false
        }
    case *ast.AssignStmt:
        fmt.Printf("asgn %s: %s - %s\n", fset.Position(n.Pos()), x.Lhs, x.Rhs)
        //tda = *x
        //tdn = n
    case *ast.ExprStmt:
        if x != nil {
            fmt.Printf("exprst %s: %s - %s\n", fset.Position(n.Pos()), x.X, "")
        }
	}
	return true
}, nil)

//var abuf bytes.Buffer
//printer.Fprint(&abuf, fset, fexp)
//fmt.Println("after\n----\n", abuf.String())

asgn /Users/ekingery/dev/exercism/go-test-runner/testrunner/testdata/concept/conditionals/conditionals_test.go:2:24: [tests] - [%!s(*ast.CompositeLit=&{0xc00050b500 98 [0xc0005574c0 0xc000557540 0xc0005575c0] 284 false})]
exprst /Users/ekingery/dev/exercism/go-test-runner/testrunner/testdata/concept/conditionals/conditionals_test.go:24:18: &{%!s(*ast.SelectorExpr=&{0xc000501ee0 0xc000501f00}) %!s(token.Pos=321) [%!s(*ast.SelectorExpr=&{0xc000501f40 0xc000501f60}) %!s(*ast.FuncLit=&{0xc00056c040 0xc00050b8c0})] %!s(token.Pos=0) %!s(token.Pos=477)} - 
asgn /Users/ekingery/dev/exercism/go-test-runner/testrunner/testdata/concept/conditionals/conditionals_test.go:25:32: [got] - [%!s(*ast.CallExpr=&{0xc00056c080 374 [0xc00056c0e0] 0 382})]
exprst /Users/ekingery/dev/exercism/go-test-runner/testrunner/testdata/concept/conditionals/conditionals_test.go:26:42: &{%!s(*ast.SelectorExpr=&{0xc00056c180 0xc00056c1a0}) %!s(token.Pos=414) [%!s(*ast.BasicLit=&{415 9 "ParseCard(%s) = %d, want %d"}) %!s(

&{0xc000501a00 0xc00050b920}

In [9]:
// This code uses the Inspect function to iterate over the AST
type testMeta struct {
    subTName    string           // name of subtest
    origTDName  string           // original test data []struct name
    newTDName   string           // new test data struct name
    TDAssign    ast.Node         // tests AssignStmt
    subTest     ast.CompositeLit // Run() function literal node
    rangeStmt   ast.RangeStmt    // test loop range statement node
} 

metadata := testMeta{
    subTName: "parse queen",
}

fset := token.NewFileSet()
f, err := parser.ParseFile(fset, "", "package main\n" + testCodeSrc, parser.ParseComments)
if err != nil {
    log.Fatal(err)
}
fexp, err := parser.ParseExpr(testCodeSrc)

ast.Inspect(fexp, func(n ast.Node) bool {
    switch x := n.(type) {
    case *ast.RangeStmt:        
        fmt.Printf("range %s:\t%s - %s\n", fset.Position(n.Pos()), x.X, x.Body)

    case *ast.BasicLit:
        if token.STRING == x.Kind && "\"parse queen\"" == x.Value {
            fmt.Printf("bl %s:\t%s - %s\n", fset.Position(n.Pos()), x.Kind, x.Value)
            //tdAssgn = cr.Parent()
            //tdPNode = cr.Parent()
            metadata.origTDName = "parse queen"
            //return false
        }
    case *ast.AssignStmt:
        if metadata.TDAssign == nil {
            //[TODO - search and assign]
            fmt.Printf("asgn %s: %s - %s\n", fset.Position(n.Pos()), x.Lhs, x.Rhs)
            metadata.TDAssign = n
        }
    }
    return true
})

fmt.Println("-----\n")
fmt.Printf("%+v\n", metadata)
fmt.Println("-----\n")
//fexp, err := parser.ParseExpr(testCodeSrc)
//fmt.Println(testCodeSrc)

asgn 2:24: [tests] - [%!s(*ast.CompositeLit=&{0xc000948ff0 98 [0xc000945440 0xc0009454c0 0xc000945540] 284 false})]
range 21:10:	tests - &{%!s(token.Pos=312) [%!s(*ast.ExprStmt=&{0xc000945800})] %!s(token.Pos=480)}
-----

{𒀸subTName:parse queen 𒀸origTDName: 𒀸newTDName: TDAssign:0xc000945600 𒀸subTest:{Type:<nil> Lbrace:0 Elts:[] Rbrace:0 Incomplete:false} 𒀸rangeStmt:{For:0 Key:<nil> Value:<nil> TokPos:0 Tok:ILLEGAL X:<nil> Body:<nil>}}
-----



7 <nil>